<a href="https://colab.research.google.com/github/UniVR-DH/ADHLab/blob/main/lecture01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crawling with Beautifulsoup4 and  Wikipedia Python APIs to create a document collection

<img src="https://drive.google.com/uc?export=view&id=1m_EMdnI5C826kgqK7r5vB4TXnB0-Wq7W" alt="Intestazione con loghi istituzionali" width="525"/>

| Docente      | Insegnamento | Anno Accademico    |
| :---        |    :----   |          ---: |
| Matteo Lissandrini      | Laboratorio Avanzato di Informatica Umanistica       | 2023/2024   |

### Installing additional packages

In [ ]:
%pip install wikipedia-api
%pip install beautifulsoup4
%pip install nltk

### Importing some basic required packages

In [ ]:
import gzip
import string
import numpy as np
import requests
import regex as re

### Crawling content with Beautifulsoup4
#### Select a webpage, download its content, parse the HTML to extract the text

In [ ]:
from bs4 import BeautifulSoup

page = requests.get('https://en.wikipedia.org/wiki/New_York_City')

# Create a BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser')

# Pull text from all instances of <p> tag within BodyText div
all_p_items = soup.find(class_='mw-body').find_all('p')
print(len(all_p_items))
print(all_p_items[0])
print(all_p_items[0].get_text())
print('    ----    ')
print(all_p_items[1])
print(all_p_items[1].get_text())

In [ ]:
punct_regex = re.compile('[{}]'.format(re.escape(string.punctuation))) # Regex matching any punctuation
space_regex = re.compile(' +') # Regex matching whitespace

text = punct_regex.sub(' ', soup.find(class_='mw-body').get_text())
text = space_regex.sub(' ', text).lower()  # convert to lowercase
lines = [
    line.strip()
    for line in text.split("\n")
    if line.strip() != "" # Skip empty lines
]
# Store lines
print(len(lines))
print(lines[0])
print(lines[1])
print(lines[1290])

In [ ]:
######
# TODO: Open the wikipedia page for New York, select a sentence, can you find at which line it appears?
######







In [ ]:
######
# TODO: Complete the code,
#   a) split a line in single words, compute word frequency
#   b) compute word frequency of all words across all lines
#
# Try out: https://docs.python.org/3/library/collections.html#collections.Counter
#
######

print(len(lines[1290].split(' ')))
words = set( w for w in lines[1290].split(' '))
print(len(words))
print(words)


#### Accessing Links in the page

In [ ]:
all_a_items = soup.find(class_='mw-body').find_all('a')
print(len(all_a_items))
for a in all_a_items:
  href = a.get('href')
  if href is not None and href.startswith('/wiki/') :
    print(href)

In [ ]:
######
# TODO: Create a dictionary of /wiki/ links, and count how many times they appear in the page, which are the top-5 most frequent links?
######





In [ ]:
######
# TODO: Pick the most frequent /wiki/ link from the above dictionary,
# download its page content and extract all links,
# do you find links in common ?
######




### Extract content from Wikipedia with the Wikipedia APIs

In [ ]:
import wikipediaapi
## EDIT Down There: put your name and email for the Wikipedia logs
wapi_text = wikipediaapi.Wikipedia('MyProjectName (name@studenti.univr.it)',
                                   'en',
                                   extract_format=wikipediaapi.ExtractFormat.WIKI)

In [ ]:
page_py = wapi_text.page('New York City')
print("Page - Exists: {}".format( page_py.exists()))
print(len(page_py.summary))
print(len(page_py.text))
print(len(page_py.langlinks))
print(len(page_py.links))

In [ ]:
print(page_py.summary[:140])
print("   ---   ")
print(page_py.text[-140:])
print("   ---   ")
print(sorted(page_py.langlinks.keys()))
print("   ---   ")
page_py_it = page_py.langlinks['it']
print(page_py_it.summary[:140])

In [ ]:
links = page_py.links
for title in sorted(links.keys()):
    if len(title) > 4 : # filter on title length to reduce output
      continue
    print("{}".format(title))

In [ ]:
test_pages = ['Addis Ababa',  'Tom Sawyer', 'Johannes Gutenberg']

In [ ]:
from urllib.parse import quote

page_queue = [ wapi_text.page(tp) for  tp in test_pages ]
page_stored = {}
page_visited = set()
max_iterations = 50

while len(page_queue) > 0 and max_iterations > 0:
  _page = page_queue.pop()

  page_stored[_page.fullurl] = _page.summary

  page_visited.add(_page.fullurl)

  print(max_iterations, _page.title, _page.fullurl)
  max_iterations = max_iterations - 1

  for next_page in _page.links.values():
    try:
      if len(next_page.title) < 6 and len(next_page.title) > 13:
        continue # skip this page

      if ':' in next_page.title :
        continue

      if next_page.fullurl in page_visited:
        continue # skip this page

      # otherwise
      page_queue.append(next_page)
    except:
      print("Error retrieving", next_page.title)


print(len(page_stored))
print(page_stored.keys())

In [ ]:
page_stored['https://en.wikipedia.org/wiki/Victoria_Falls_Airport']

In [ ]:
######
# TODO: Create the bag of words for all page summaries, remember to transform the text in lowercase and remove punctuation
######

#### The following declaration extract unparsed HTML instead of already parsed text

In [ ]:
wapi_html = wikipediaapi.Wikipedia('MyProjectName (name@studenti.univr.it)',
                              'en',
                              extract_format=wikipediaapi.ExtractFormat.HTML)
page_py = wapi_text.page('New York City')
print("Page - Exists: {}".format( page_py.exists()))
print(len(page_py.summary))


### Stemming nad lemmatization

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer, SnowballStemmer, WordNetLemmatizer

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

In [ ]:
# Initialize Python porter stemmer
ps = PorterStemmer()
sn = SnowballStemmer("english")

example_sentence = "Programming is an art and a job. Python programmers often tend to like programming in python because it's like english. This is a better language than many ohters an incredibly useful property that makes things easier. We call people who program in python pythonistas."

# Remove punctuation
example_sentence_no_punct = example_sentence.lower().translate(str.maketrans("", "", string.punctuation))

# Create tokens
word_tokens = word_tokenize(example_sentence_no_punct)

# Perform stemming
print("{0:20}{1:20}".format("--Word--","--Stem--"))
for word in word_tokens:
    print ("{0:20}{1:20}{2:20}".format(word, ps.stem(word), sn.stem(word)))


In [ ]:

# Initialize wordnet lemmatizer
wnl = WordNetLemmatizer()

# wn.VERB
# wn.ADV
# wn.NOUN

# Perform lemmatization
print("{0:20}{1:20}".format("--Word--","--Lemma--"))
for word in word_tokens:
   print ("{0:20}{1:20}".format(word, wnl.lemmatize(word, pos=wordnet.ADJ))) # <- lemmatize as if they are all adjectives


In [ ]:
######
# TODO: Text stemming and lemmatization with a wikipedia page summary
######

